In [648]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [649]:
circuits = pd.read_csv('circuits.csv',sep=',')

In [650]:
circuits.drop(columns = ['url','alt'], axis = 1, inplace=True)

In [651]:
circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500


In [652]:
fig = px.scatter_geo(circuits,lat='lat', lon='lng', hover_name="circuitId", projection="natural earth")
fig.update_layout(title = 'Distribution of Historical Races', title_x=0.5)
fig.show()

## Identify 2023 Circuits and Calculate Pairwise Distances

In [653]:
selected_circuits_2023 = [24,1,73,3,9,7,39,21,11,14,22,79,77,18,15,4,78,17,44]

In [654]:
len(selected_circuits_2023)

19

In [655]:
# Data for chosen circuits

circuits_2023 = circuits[circuits.circuitId.isin(selected_circuits_2023)]

circuits_2023 = circuits_2023.replace(list(circuits_2023['circuitId']), list(range(0,len(circuits_2023))))

circuits_2023

,circuitId,circuitRef,name,location,country,lat,lng
0,0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.96800
2,1,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.51060
3,2,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.5700,2.26111
6,3,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5000,-73.52280
8,4,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694
10,5,hungaroring,Hungaroring,Budapest,Hungary,47.5789,19.24860
13,6,monza,Autodromo Nazionale di Monza,Monza,Italy,45.6156,9.28111
14,7,marina_bay,Marina Bay Street Circuit,Marina Bay,Singapore,1.2914,103.86400
16,8,shanghai,Shanghai International Circuit,Shanghai,China,31.3389,121.22000
17,9,interlagos,Autódromo José Carlos Pace,São Paulo,Brazil,-23.7036,-46.69970


In [656]:
import plotly.graph_objects as go
import pandas as pd
import geopy.distance
import itertools

In [657]:
# Create a list of combinations from circuits_2023 

combinations = itertools.combinations(list(range(0,len(circuits_2023))), 2)
combinations = list(combinations)
len(combinations)

171

In [658]:
fig = go.Figure()

# Draw every possible connection between circuits


for i in range(len(combinations)):
    circuit_id_1 = combinations[i][0],
    circuit_id_2 = combinations[i][1],
    fig.add_trace(
        go.Scattergeo(
            # for each combination, find longitude and latitude coordinates
            lon = [float(circuits_2023[(circuits_2023['circuitId']==circuit_id_1)==True]['lng']), float(circuits_2023[(circuits_2023['circuitId']==circuit_id_2)==True]['lng'])],
            lat = [float(circuits_2023[(circuits_2023['circuitId']==circuit_id_1)==True]['lat']), float(circuits_2023[(circuits_2023['circuitId']==circuit_id_2)==True]['lat'])],
            mode = 'lines',
            line = dict(width = 1,color = 'turquoise'),
            opacity = 0.5
                    )
                )

# Add points corresponding to circuit locations

fig.add_trace(go.Scattergeo(
    lon = circuits_2023['lng'],
    lat = circuits_2023['lat'],
    hoverinfo = 'text',
    text = circuits_2023['name'],
    mode = 'markers',
    marker = dict(
        size = 7,
        color = 'crimson',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

# Format output

fig.update_layout(
    title_text = 'Flight Paths Between Confirmed 2023 Formula 1 Circuits',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type="orthographic",
        # projection_type = 'natural earth',
        showland = True,
        showcountries=True,
        #landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'white',
    ),
)

In [659]:
# Create list of distances between pairs of circuits

dist_list = combinations

for i in range(len(combinations)):
    # (lat, lng)
    coords_1 = (circuits_2023[circuits_2023['circuitId'] == combinations[i][0]]['lat'],
                circuits_2023[circuits_2023['circuitId'] == combinations[i][0]]['lng'])
    coords_2 = (circuits_2023[circuits_2023['circuitId'] == combinations[i][1]]['lat'],
                circuits_2023[circuits_2023['circuitId'] == combinations[i][1]]['lng'])
    
    coords_1 = (float(coords_1[0]), float(coords_1[1]))
    coords_2 = (float(coords_2[0]), float(coords_2[1]))
    
    # dist_list is a list of triples
    
    dist_list[i] = (combinations[i][0], combinations[i][1], geopy.distance.geodesic(coords_1, coords_2).km)
    

In [660]:
# dist_list

In [661]:
import mlrose
import numpy as np

In [662]:
# Initialize fitness function object using dist_list
fitness_dists = mlrose.TravellingSales(distances = dist_list)

In [663]:
problem_fit = mlrose.TSPOpt(length = len(circuits_2023), fitness_fn = fitness_dists, maximize=False)

In [664]:
# Solve problem using the genetic algorithm

best_state, best_fitness = mlrose.genetic_alg(problem_fit)

print('The best state found is: ', best_state)

print('The fitness at the best state is: ', best_fitness)

The best state found is:  [18  3  6 11  0  7 17  9  2  5  4 12  1 15  8 16 10 13 14]
The fitness at the best state is:  100869.69473912234


In [665]:
circuits_2023

,circuitId,circuitRef,name,location,country,lat,lng
0,0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.96800
2,1,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.51060
3,2,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.5700,2.26111
6,3,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5000,-73.52280
8,4,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694
10,5,hungaroring,Hungaroring,Budapest,Hungary,47.5789,19.24860
13,6,monza,Autodromo Nazionale di Monza,Monza,Italy,45.6156,9.28111
14,7,marina_bay,Marina Bay Street Circuit,Marina Bay,Singapore,1.2914,103.86400
16,8,shanghai,Shanghai International Circuit,Shanghai,China,31.3389,121.22000
17,9,interlagos,Autódromo José Carlos Pace,São Paulo,Brazil,-23.7036,-46.69970


In [666]:
for i in range((len(best_state)-1)):
    circuit_id_1 = int(best_state[i]),
    circuit_id_2 = int(best_state[i+1]),
    fig.add_trace(
        go.Scattergeo(
            # for each combination, find longitude and latitude coordinates
            lon = [float(circuits_2023[(circuits_2023['circuitId']==circuit_id_1)==True]['lng']), float(circuits_2023[(circuits_2023['circuitId']==circuit_id_2)==True]['lng'])],
            lat = [float(circuits_2023[(circuits_2023['circuitId']==circuit_id_1)==True]['lat']), float(circuits_2023[(circuits_2023['circuitId']==circuit_id_2)==True]['lat'])],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = 1
                    )
                )
    
# Add points corresponding to circuit locations

fig.add_trace(go.Scattergeo(
    lon = circuits_2023['lng'],
    lat = circuits_2023['lat'],
    hoverinfo = 'text',
    text = circuits_2023['name'],
    mode = 'markers',
    marker = dict(
        size = 7,
        color = 'crimson',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))
    
fig.update_layout(
    title_text = 'Flight Paths Between Confirmed 2023 Formula 1 Circuits',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type="natural earth",
        showland = True,
        showcountries=True,
        #landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'white',
    ),
)
fig.show()

In [689]:
len(best_state)
a = circuits_2023[circuits_2023['circuitId'] == best_state[i]]['circuitRef']
str(a)

'38    zandvoort\nName: circuitRef, dtype: object'

In [673]:
for i in len(best_state):
    circuits_2023[circuits_2023['circuitId'] == best_state[i]]

TypeError: 'int' object is not iterable